# 지난 시간 복습

1. Variable
2. Function
3. Numerical Differential

In [195]:
import numpy as np

In [196]:
class Variable(object):
    def __init__(self, data):
        self.data = data

class Function(object):
    def __call__(self, var):
        y = self.forward(var)
        return Variable(y)
    
    def forward(self):
        raise NotImplementedError
    
def numerical_diff(f, x, eps=1e-4):
    x = x.data
    return (f(Variable(x + eps)).data - f(Variable(x - eps)).data) / (2 * eps)



Let $f(x) = 10x - 5$  
then, $f'(x) = 10$  
  
Now, I'll check that numerical differential can approximate the real differential.

In [197]:
class f(Function):
    def forward(self, data):
        return 10 * data.data - 5
    
def approx(data):
    return round(data)
    
real_diff = 10
data = Variable(np.array(10))
num_diff = numerical_diff(f(), data)
print(num_diff)
print(approx(num_diff) == real_diff)

10.000000000047748
True


In [198]:
class f(Function):
    def forward(self, data):
        return np.exp(data.data**2) ** 2
    
data = Variable(np.array(0.5))
dy = numerical_diff(f(), data)
print(dy)

3.2974426293330694


이번 주에는 총 3가지를 배운다.  

1. Backpropagation-Theory
2. Manual Backpropagation
3. Automatic Backpropagation
  
--- 
개인적인 공부 
1. Python magic method
2. Python decorator

# Not clone coding

책 내용을 1번만 읽고, 기억나는 기능들을 구현  

역전파의 내용을 잘 이해하지 않고, 따라해서 관계식이 잘못됨.

In [199]:
class Variable(object):
    def __init__(self, data: Variable):
        self.data: Variable = data
        self.grad: Variable = None
        self.parent: Function = None

    def set_parent(self, parent: Function):
        self.parent = parent

    def backward(self):
        f = self.parent
        if f is not None:
            dx = f.input
            dy = f.backward()
            dx.grad = dy
            dx.backward()

class Function(object):
    def __call__(self, var):
        self.input: Variable = var
        x = var.data

        y = self.forward(x)
        y = Variable(y)
        self.output = y

        y.set_parent(self)
        return y
    
    def forward(self):
        return NotImplementedError
    
    def backward(self):
        return NotImplementedError
    
class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self):
        return Variable(np.exp(self.output.grad.data))
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self):
        return Variable(2 * self.output.grad.data)

In [200]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = Variable(np.array(1.0))
y.backward()
print(x.grad.data)

14.7781121978613


# 1. Backpropagation-Theory

전 장(step. 4)의 마지막에서 Numerical differential의 단점 (**_높은 계산 복잡도_** , **_오차_** )을 언급하면서 끝나서,  
오차역전파가 미분을 위한 방법론이라고 착각했다.  
미분을 위한 방법보다는 미분한 값을 어떻게 **_효율적으로 퍼뜨릴 것인가_** 에 대한 방법론인 것 같다.  

## 오차 역전파에 대한 이해
$f(g) = e^{g}, g(x) = x^{2}$ 이라고 하자. x=2에서 f의 미분값을 구하라.

**_전통적인 미분)_** 
  
  $\frac{df}{dx} = \frac{dg}{dx}\frac{df}{dg}$
  
  $\frac{dg}{dx} = 2x, \frac{df}{dg} = e^{g}, \frac{df}{dx} = 2xe^{h} = 2xe^{x^{2}}$

  합성 함수의 미분법에 의해서도,

  $\frac{df}{dx} = 2xe^{x^{2}}$

  $\therefore \frac{df}{dx}|_{x=2} = 4e^{4}$
  
  
**_수치 미분-중앙 차분)_**

  $\frac{e^{(2+0.0001)^{2}} - e^{(2-0.0001)^{2}}}{2 \times 0.0001} = 218.3926081... \approx 4e^{4}$

**_역전파)_**  

  $g'(x) = 2x, f'(g) = e^{g}$  
    
  현재 미분하는 함수를 $cur$, 이전 함수를 $prev$ 라고 해보자. 그렇다면, 현재 함수의 미분값($gy$)는 다음과 같다.  
  
  $cur.gy = prev.gy \times f'(cur.input)$

  $2 \rightarrow g(x) \rightarrow 4 \rightarrow f(g) \rightarrow e^{4}$    
  $\space\space\space\space\space\searrow\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\space\searrow$  
  $4e^{4} \leftarrow g'(x) \leftarrow e^{4} \leftarrow f'(g) \leftarrow 1$  





# 2. Manual Backpropagation

In [201]:
class Variable(object):
    def __init__(self, data):
        self.data = data
        self.grad = None

class Function(object):
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        return output
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [202]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


# 3. Automatically Backpropagation

In [203]:
class Variable(object):
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

class Function(object):
    def __call__(self, var):
        x = var.data
        y = self.forward(x)
        y = Variable(y)
        y.set_creator(self)
        self.input = var
        self.output = y
        return y
    
    def forward(self):
        return NotImplementedError()
    
    def backward(self):
        return NotImplementedError()
    
class Square(Function):
    def forward(self, data):
        return data ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, data):
        return np.exp(data)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [204]:

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## Additional experiments

아래 1가지 내용을 확인
1. 입력이 벡터, 행렬이 되어도 잘 작동하나

### 1. 입력이 벡터가 되어도 잘 작동하나

In [205]:
# Vector-input test
x = Variable(np.array([0.5, 1.0, 2.0]))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

[3.29744254e+00 2.95562244e+01 2.38476639e+04]


In [206]:
# Matrix-input test
x = Variable(np.array([[0.5, 1.0, 2.0],[2.1, 2.5, 3.0]]))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

[[3.29744254e+00 2.95562244e+01 2.38476639e+04]
 [5.68534229e+04 2.68337287e+06 7.87919630e+08]]
